In [1]:
import os 
import re
import pickle

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score

# Utils

In [2]:
def crawl_paths(experiment_name):
    """
    Crawls the paths and gathers the configuration and results files created from the 
    """
    paths = []
    for (dirpath, dirnames, filenames) in os.walk(experiment_name):
        files = [os.path.join(dirpath, file) for file in filenames]
        paths.extend(files)
    configs = [path for path in paths if 'config.yaml' in path]
    xvals = [path for path in paths if 'xval' in path and 'results' in path]

    full_paths = {}        
    date_re = re.compile('(.*202[0-9]_[0-9]+_[0-9]+_[0-9]+/)')
    model_re = re.compile('.*logger/(.*)/')
    fold_re = re.compile('.*f([\-0-9]+)')
    for config_path in configs:
        try:
            experiment_date_path = date_re.findall(config_path)[0]
            exp_xvals = [xval for xval in xvals if experiment_date_path in xval]
            full_paths[experiment_date_path] = {
                'config': config_path,
                'xval': exp_xvals[0],
            }
        except IndexError:
            continue

    return full_paths

In [3]:
def look_demographics(path):
    paths_lstm = crawl_paths(path)

    with open('../../data/beerslaw/features/raw/rankings.pkl', 'rb') as fp:
        post_test = pickle.load(fp)
        post_test = post_test.set_index('username')
    demographics = ['gender', 'language', 'field', 'year']
    scores = {dem: {} for dem in demographics}
    
    seed = re.compile('raw_full/([0-9]+\_[0-9]+\_[0-9]+\_[0-9]+)/')
    cluster_iter = re.compile('/([0-9]+\_[0-9]+)/')
    seed_data = {}
    
    for token in paths_lstm:
        with open(paths_lstm[token]['xval'], 'rb') as fp:
            res = pickle.load(fp)
        try:
            with open(paths_lstm[token]['config'], 'rb') as fp:
                config = pickle.load(fp)
        except pickle.UnpicklingError:
            with open(paths_lstm[token]['config'], 'r') as fp:
                config = yaml.load(fp, Loader=yaml.FullLoader)
            
        path_seed = seed.findall(token)[0] + '_' + cluster_iter.findall(token)[0]
            
        idd = config['id_dictionary']

        test_indices = []
        test_indexes = []
        y_truths = []
        y_preds = []
        y_probas = []
        learner_ids = []
        genders = []
        languages = []
        fields = []
        years = []
        for fold in range(10):
            test_indix = list(res[fold]['test_indices'])
            test_index = list(res[fold]['test_index'])
            x_test = [res['x'][idx] for idx in test_index]
            longenough = [i for i in range(len(x_test)) if len(x_test[i]) > 10]
            test_index = [test_index[i] for i in longenough]
            
            y_truth = [res['y'][idx] for idx in test_index]
            y_pred = list(res[fold]['y_pred'])
            y_pred = [y_pred[idx] for idx in longenough]
            y_proba = list(res[fold]['y_proba'])
            y_proba = [y_proba[idx] for idx in longenough]
            lid = [idd['sequences'][idx]['learner_id'] for idx in test_indix]
            lid = [lid[idx] for idx in longenough]
            gender = [post_test.loc[liid]['gender'] for liid in lid]
            language = [post_test.loc[liid]['language'] for liid in lid]
            field = [post_test.loc[liid]['field'] for liid in lid]
            year = [post_test.loc[liid]['year'] for liid in lid]

            test_indices = test_indices + test_indix
            test_indexes = test_indexes + test_index
            y_truths = y_truths + y_truth
            y_preds = y_preds + y_pred
            y_probas = y_probas + y_proba
            learner_ids = learner_ids + lid
            genders = genders + gender
            languages = languages + language
            fields = fields + field
            years = years + year

        data = {
            'y_test': y_truths,
            'y_pred': y_preds,
            'y_proba': y_probas,
            'learner_ids': learner_ids,
            'gender': genders, 
            'language': languages,
            'field': fields,
            'year': years
        }
        
        seed_data[path_seed] = data
    return seed_data

In [4]:
def true_positive(y_test, y_pred):
    positive = [i for i in range(len(y_test)) if y_test[i] == 1]
    yt = np.array([y_test[i] for i in positive])
    yp = np.array([y_pred[i] for i in positive])
    return sum(yt == yp) / len(positive)

def false_positive(y_test, y_pred):
    fpositive = [i for i in range(len(y_test)) if y_test[i] == 0]
    yt = np.array([y_test[i] for i in fpositive])
    yp = np.array([y_pred[i] for i in fpositive])
    return sum(yt != yp) / len(fpositive)

def pred_positive(y_test, y_pred):
    positive = [yy for yy in y_pred if yy == 1]
    return len(positive) / len(y_test)

def pred_roc(y_test, y_pred):
    return roc_auc_score(y_test, y_pred)
    
def print_equalised_odds_summary_per_demo(seed_data, demographics): 
    seed_list = list(seed_data.keys())
    demographics_list = np.unique(seed_data[seed_list[0]][demographics])
    
    demo_seeds = {}
    demos = []
    for demo in demographics_list:
        demos.append(demo)
        demo_seeds[demo] = {}
        seed_tps = []
        seed_fps = []
        seed_pps = []
        seed_rocs = []
        for seed in seed_data:
            demo_indices = [i for i in range(len(seed_data[seed][demographics])) if seed_data[seed][demographics][i] == demo]
            demo_ytest = [seed_data[seed]['y_test'][i] for i in demo_indices]
            demo_ypred = [seed_data[seed]['y_pred'][i] for i in demo_indices]
            tp = true_positive(demo_ytest, demo_ypred)
            fp = false_positive(demo_ytest, demo_ypred)
            pp = pred_positive(demo_ytest, demo_ypred)
            roc = pred_roc(demo_ytest, demo_ypred)
            seed_tps.append(tp)
            seed_fps.append(fp)
            seed_pps.append(pp)
            seed_rocs.append(roc)
            
            demo_seeds[demo][seed] = {
                'tp': tp,
                'fp': fp,
                'pp': pp,
                'roc': roc
            }

        demo_seeds[demo]['tps'] = np.mean(seed_tps)
        demo_seeds[demo]['fps'] = np.mean(seed_fps)
        demo_seeds[demo]['pps'] = np.mean(seed_pps)
        demo_seeds[demo]['rocs'] = np.mean(seed_rocs)
        
    demos, tps, fps, pps, rocs = [], [], [], [], []
    for demo in demo_seeds:
        demos.append(demo)
        tps.append(demo_seeds[demo]['tps'])
        fps.append(demo_seeds[demo]['fps'])
        pps.append(demo_seeds[demo]['pps'])
        rocs.append(demo_seeds[demo]['rocs'])
        print('  attribute: {} ({})'.format(demo, len(demo_indices)))
        print('      tp: {}\n      fp: {}\n      pp: {}\n      roc: {}\n'.format(
            demo_seeds[demo]['tps'],
            demo_seeds[demo]['fps'],
            demo_seeds[demo]['pps'],
            demo_seeds[demo]['rocs']
        ))
            
        
        
    print('  Sorted by True Positive')
    pos_indices = np.argsort(tps)
    demos = [demos[idx] for idx in pos_indices]
    sorted_tps = [tps[idx] for idx in pos_indices]
    print_tps = ['{} ({})'.format(demos[i], sorted_tps[i]) for i in range(len(demos))]
    print('     ', print_tps)
    print()
    
    print('  Sorted by False Positives')
    neg_indices = np.argsort(fps)
    demos = [demos[idx] for idx in neg_indices]
    sorted_tns = [tps[idx] for idx in neg_indices]
    print_tps = ['{} ({})'.format(demos[i], sorted_tps[i]) for i in range(len(demos))]
    print('     ', print_tps)
    print()
    
    print('  Sorted by Predicted Positive')
    pps_indices = np.argsort(pps)
    demos = [demos[idx] for idx in pps_indices]
    sorted_pps = [pps[idx] for idx in pps_indices]
    print_pps = ['{} ({})'.format(demos[i], sorted_pps[i]) for i in range(len(demos))]
    print('     ', print_pps)
    print()
    
    print('  Sorted by ROC')
    rocs_indices = np.argsort(rocs)
    demos = [demos[idx] for idx in rocs_indices]
    sorted_rocs = [rocs[idx] for idx in rocs_indices]
    print_rocs = ['{} ({})'.format(demos[i], sorted_rocs[i]) for i in range(len(demos))]
    print('     ', print_rocs)
    print()
    
def print_equalised_odds_summary(seed_data):
    for demographics in ['gender', 'language', 'field', 'year']:
        print('='*100)
        print('ATTRIBUTE: {}'.format(demographics))
        print_equalised_odds_summary_per_demo(seed_data, demographics)
        print()
        print()
        
def average_across_seeds(seed_data):
    seeds = list(seed_data.keys())
    ypreds = []
    yprobas = []
    ytests = []
    genders = []
    languages = []
    fields = []
    years = []
    lids = []
    for learner_id in seed_data[seeds[0]]['learner_ids']:
        ypred = np.sum([seed_data[seed]['y_pred'][seed_data[seed]['learner_ids'].index(learner_id)] for seed in seeds])
        ypred /= len(seeds)
        ypred = int(ypred)
        ypreds.append(ypred)
        
        yproba = np.sum([seed_data[seed]['y_proba'][seed_data[seed]['learner_ids'].index(learner_id)] for seed in seeds])
        yproba /= len(seeds)
        yprobas.append(yproba)
        
        ytests.append(seed_data[seeds[0]]['y_test'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        genders.append(seed_data[seeds[0]]['gender'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        languages.append(seed_data[seeds[0]]['language'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        fields.append(seed_data[seeds[0]]['field'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        years.append(seed_data[seeds[0]]['year'][seed_data[seeds[0]]['learner_ids'].index(learner_id)])
        lids.append(learner_id)
        
    seed_data['all_data'] = {
        'y_test': ytests,
        'y_pred': ypreds,
        'y_proba': yprobas,
        'learner_ids': lids,
        'gender': genders, 
        'language': languages,
        'field': fields,
        'year': years
    }
    return seed_data
        
        



In [7]:
path = '../../data/beerslaw/algos_results/lstm'
seed_data = look_demographics(path)
eo = print_equalised_odds_summary(seed_data)

ATTRIBUTE: gender
  attribute: 1 (7)
      tp: 0.613030303030303
      fp: 0.16437500000000002
      pp: 0.3471604938271605
      roc: 0.7243276515151515

  attribute: 2 (7)
      tp: 0.6028301886792452
      fp: 0.24545454545454543
      pp: 0.4208333333333334
      roc: 0.6786878216123499

  attribute: 3 (7)
      tp: 0.0
      fp: 0.10555555555555554
      pp: 0.07916666666666666
      roc: 0.44722222222222224

  attribute: 4 (7)
      tp: 0.5166666666666667
      fp: 0.046666666666666676
      pp: 0.18095238095238095
      roc: 0.735

  Sorted by True Positive
      ['3 (0.0)', '4 (0.5166666666666667)', '2 (0.6028301886792452)', '1 (0.613030303030303)']

  Sorted by False Positives
      ['1 (0.0)', '2 (0.5166666666666667)', '3 (0.6028301886792452)', '4 (0.613030303030303)']

  Sorted by Predicted Positive
      ['3 (0.07916666666666666)', '4 (0.18095238095238095)', '1 (0.3471604938271605)', '2 (0.4208333333333334)']

  Sorted by ROC
      ['1 (0.44722222222222224)', '4 (0.67868782

In [12]:
rocs = []
for key in seed_data:
    r = roc_auc_score(seed_data[key]['y_test'], [rr[1] for rr in seed_data[key]['y_proba']])
    rocs.append(r)
print(np.mean(rocs))

0.7587874587874589


In [13]:
path = '../../data/beerslaw/algos_results/ts_attention/'
seed_data = look_demographics(path)
eo = print_equalised_odds_summary(seed_data)

ATTRIBUTE: gender
  attribute: 1 (7)
      tp: 0.6057575757575756
      fp: 0.21187499999999998
      pp: 0.37234567901234567
      roc: 0.6969412878787878

  attribute: 2 (7)
      tp: 0.6069182389937107
      fp: 0.27909090909090906
      pp: 0.4399691358024692
      roc: 0.663913664951401

  attribute: 3 (7)
      tp: 0.0
      fp: 0.5777777777777776
      pp: 0.43333333333333335
      roc: 0.2111111111111111

  attribute: 4 (7)
      tp: 0.625
      fp: 0.09333333333333334
      pp: 0.24523809523809523
      roc: 0.7658333333333334

  Sorted by True Positive
      ['3 (0.0)', '1 (0.6057575757575756)', '2 (0.6069182389937107)', '4 (0.625)']

  Sorted by False Positives
      ['4 (0.0)', '3 (0.6057575757575756)', '1 (0.6069182389937107)', '2 (0.625)']

  Sorted by Predicted Positive
      ['2 (0.24523809523809523)', '4 (0.37234567901234567)', '1 (0.43333333333333335)', '3 (0.4399691358024692)']

  Sorted by ROC
      ['1 (0.2111111111111111)', '4 (0.663913664951401)', '2 (0.696941287

In [15]:
rocs = []
for key in seed_data:
    r = roc_auc_score(seed_data[key]['y_test'], [rr[1] for rr in seed_data[key]['y_proba']])
    rocs.append(r)
print(np.mean(rocs))

0.7519414519414519


In [14]:
import yaml
yaml_path = '../../data/beerslaw/algos_results/ts_attention/simplestate_secondslstm/binconcepts/ts_attention/raw_full/2022_03_02_0/config.yaml'
with open(yaml_path, 'r') as fp:
    yy = yaml.load(fp, Loader=yaml.FullLoader)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/beerslaw/algos_results/ts_attention/simplestate_secondslstm/binconcepts/ts_attention/raw_full/2022_03_02_0/config.yaml'

In [38]:
seed_data.keys()

dict_keys(['2022_03_03_7_15_30', '2022_03_03_0_15_30', '2022_03_03_9_15_30', '2022_03_03_11_15_30', '2022_03_03_8_15_30', '2022_03_03_1_15_30', '2022_03_03_6_15_30', '2022_03_03_10_15_30', '2022_03_03_12_15_30', '2022_03_03_3_15_30', '2022_03_03_4_15_30', '2022_03_03_14_15_30', '2022_03_03_13_15_30', '2022_03_03_5_15_30', '2022_03_03_2_15_30', '2022_03_03_7_45_61', '2022_03_03_0_45_61', '2022_03_03_9_45_61', '2022_03_03_11_45_61', '2022_03_03_8_45_61', '2022_03_03_1_45_61', '2022_03_03_6_45_61', '2022_03_03_10_45_61', '2022_03_03_12_45_61', '2022_03_03_15_45_61', '2022_03_03_3_45_61', '2022_03_03_4_45_61', '2022_03_03_14_45_61', '2022_03_03_13_45_61', '2022_03_03_5_45_61', '2022_03_03_2_45_61', '2022_03_03_7_30_45', '2022_03_03_0_30_45', '2022_03_03_9_30_45', '2022_03_03_11_30_45', '2022_03_03_8_30_45', '2022_03_03_1_30_45', '2022_03_03_6_30_45', '2022_03_03_10_30_45', '2022_03_03_12_30_45', '2022_03_03_3_30_45', '2022_03_03_4_30_45', '2022_03_03_14_30_45', '2022_03_03_13_30_45', '2022

In [37]:
roc_auc_score(seed_data['all_data']['y_test'], seed_data['all_data']['y_proba'])

0.5035595035595035

In [40]:
roc_auc_score(seed_data['2022_03_03_7_15_30']['y_test'], [yp[1] for yp in seed_data['2022_03_03_7_15_30']['y_proba']])

0.7484407484407485

In [6]:
seed_data.keys()

dict_keys(['60', '69', '5', '3', '75', '16', '64', '0', '96'])

In [7]:
seed_data['60'].keys()

dict_keys(['y_test', 'y_pred', 'y_proba', 'learner_ids', 'gender', 'language', 'field', 'year'])

In [69]:
seed_data['60']['learner_ids'].index('2hr6mkdc')

3

In [5]:
a = ['1', 3, 4]

In [7]:
a.remove(3)

In [8]:
a

['1', 4]